Important Libraries

In [1]:
import requests
import pandas as pd

### API

Get info via API

In [9]:
def get_info(siren, url2):
    url = f"https://siva-integ1.cegedim.cloud/apim/api-asso/api/structure/{siren}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: Failed to retrieve data from the API \n Code: {response.status_code}")
    
    add = [elem['adresse'] for elem in data['etablissements']['etablissement']]

    return {'nom': data['identite']['nom'], 'activites': data['activites']['objet'], 'lib_activité': data['activites']['lib_activite_principale'], 'nb_etablissements': data['etablissements']['nb_actifs'], 'url': url2, 'adresses': add}

In [10]:
info_sp = get_info(784228090, 'https://www.secourspopulaire.fr/')
info_re = get_info(839628716, 'https://www.regleselementaires.com/')
info_es = get_info(317236248, 'https://www.emmaus-solidarite.org/')

In [11]:
print(info_sp)

{'nom': 'SECOURS POPULAIRE FRANCAIS', 'activites': "soutenir matériellement moralement juridiquement les victimes de l'arbitraire de la misère de la faim ainsi que leurs familles", 'lib_activité': 'Action sociale sans hébergement n.c.a.', 'nb_etablissements': 1, 'url': 'https://www.secourspopulaire.fr/', 'adresses': [{'num_voie': 10, 'type_voie': 'RUE', 'voie': 'PAUL BERT', 'cp': 90000, 'commune': 'BELFORT', 'code_insee': 90010, 'code_zrr': 'NC - Commune non classée'}, {'cplt_1': 'LE DURANDAL', 'num_voie': 131, 'voie': 'DE PARIS', 'cp': 83200, 'commune': 'TOULON', 'code_insee': 83137, 'code_zrr': 'NC - Commune non classée'}, {'num_voie': 24, 'voie': 'D AIX', 'cp': 13120, 'commune': 'GARDANNE', 'code_insee': 13041, 'code_zrr': 'NC - Commune non classée'}, {'num_voie': 47, 'voie': 'VOGEL', 'cp': 80000, 'commune': 'AMIENS', 'code_insee': 80021, 'code_zrr': 'NC - Commune non classée'}, {'num_voie': 141, 'voie': 'JEAN JAURES', 'cp': 90000, 'commune': 'BELFORT', 'code_insee': 90010, 'code_zr

In [12]:
data = [info_sp, info_re, info_es]

df = pd.DataFrame(data)
df.head()

,nom,activites,lib_activité,nb_etablissements,url,adresses
0,SECOURS POPULAIRE FRANCAIS,soutenir matériellement moralement juridiqueme...,Action sociale sans hébergement n.c.a.,1,https://www.secourspopulaire.fr/,"[{'num_voie': 10, 'type_voie': 'RUE', 'voie': ..."
1,REGLES ELEMENTAIRES,aider les personnes les plus démunies à accéde...,Autres organisations fonctionnant par adhésion...,1,https://www.regleselementaires.com/,"[{'cplt_2': 'LA CITE AUDACIEUSE', 'num_voie': ..."
2,EMMAUS SOLIDARITE,développer des actions de solidarité partagées...,Hébergement social pour adultes et familles en...,11,https://www.emmaus-solidarite.org/,"[{'num_voie': 10, 'type_voie': 'RUE', 'voie': ..."


In [13]:
df.to_csv('assos.csv', index=False)

### BeautifulSoup

In [68]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

##### Creating DataFrames for the tables

In [69]:
assos = pd.DataFrame(columns=['name', 'activity', 'siren', 'url'])
actions = pd.DataFrame(columns=['asso', 'title', 'url', 'all_text'])

##### Secours Populaires

In [70]:
urls = ['https://www.secourspopulaire.fr/', 'https://www.regleselementaires.com/', 'https://www.emmaus-solidarite.org/', 'https://www.wwf.fr/']

Get list of urls for actions

In [71]:
response = requests.get(urls[0])
content = response.content

soup = BeautifulSoup(content, 'html.parser')

hrefs = []
titles = []
asso = []

hrefs2 = []
uls = soup.find_all('ul', class_='header__sub-menu__list-content-level-1')
for ul in uls:
    for a in ul.find_all('a'):
        href = a.get('href')
        
        if href and "nos-actions" in href:
            hrefs.append(href)
            titles.append(a.text)
            asso.append('Secours Populaire')
            hrefs2.append(href)

hrefs[-2] = 'https://www.secourspopulaire.fr/nos-actions/estime-de-soi'
hrefs2[-2] = 'https://www.secourspopulaire.fr/nos-actions/estime-de-soi'

for href in hrefs:
    print(href)
for title in titles:
    print(title)


print(len(hrefs))
print(len(titles))


https://www.secourspopulaire.fr/nos-actions/urgences-climatiques/
https://www.secourspopulaire.fr/nos-actions/conflits/
https://www.secourspopulaire.fr/nos-actions/aide-migrants-refugies/
https://www.secourspopulaire.fr/nos-actions/aide-sans-abri/
https://www.secourspopulaire.fr/nos-actions/acces-aux-vacances/
https://www.secourspopulaire.fr/nos-actions/accueil-ecoute/
https://www.secourspopulaire.fr/nos-actions/aide-alimentaire/
https://www.secourspopulaire.fr/nos-actions/aide-materielle/
https://www.secourspopulaire.fr/nos-actions/acces-aux-droits/
https://www.secourspopulaire.fr/nos-actions/acces-eau-energie/
https://www.secourspopulaire.fr/nos-actions/prevention-soins/
https://www.secourspopulaire.fr/nos-actions/fracture-numerique/
https://www.secourspopulaire.fr/nos-actions/culture-sports-loisirs/
https://www.secourspopulaire.fr/nos-actions/acces-education/
https://www.secourspopulaire.fr/nos-actions/insertion-socio-professionnelle/
https://www.secourspopulaire.fr/nos-actions/esti

Getting the text from those urls

In [72]:
texts = []
for url in hrefs2:
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, 'html.parser')

    main = soup.find('main') 
    if main:
        paragraphs = main.find_all('p')

        para_list = []
        for p in paragraphs:
            #if 'thematic__title' not in p.get('class', []):
            para_list.append(p.text)
        
        texts.append(" ".join(para_list))

print(texts)
print(len(texts))

['Protéger Dès lors qu’une catastrophe climatique se produit, il est nécessaire d’apporter une aide d’urgence aux victimes. Les personnes précaires sont d’autant plus impactées par ces drames. Les bénévoles du Secours populaire se mobilisent pour répondre à leurs besoins. L’aide d’urgence prend ensuite la forme d’un accompagnement dans la durée pour aider ces familles à renouer avec leur quotidien. 21,5 millions  de réfugiés climatiques dans le monde ont été recensés en 2022. (Source : Haut Commissariat de l’ONU pour les réfugiés) Inondations, tempêtes, fortes intempéries, grand froid, catastrophes naturelles ou industrielles, incendies… Les conséquences de ces drames sont plus terribles encore chez les familles en situation de pauvreté. Dès qu’un drame humanitaire se produit, un appel aux dons est lancé par le Secours populaire et des aides d’urgence sont apportées au plus vite aux sinistrés. Face aux urgences, le Secours populaire agit en lien avec les pouvoirs publics, les structure

##### Règles Elémentaires

Get list of urls for actions

In [73]:
response = requests.get(urls[1])
content = response.content

hrefs2 = []

soup = BeautifulSoup(content, 'html.parser')

for a in soup.find_all('a', class_="hover:font-bold hover:border-b-2 border-white"):
    href = a.get('href')
    if href and 'action' in href:
        hrefs.append(urls[1] + href[1::])
        hrefs2.append(urls[1] + href[1::])
        titles.append(a.text)
        asso.append('Règles Elémentaires')

print(len(hrefs))
print(len(titles))


20
20


Getting the text from those urls

In [74]:
for url in hrefs2:
    
    text = []
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, 'html.parser')

    # Extracting all paragraphs from the page
    resume = soup.find('div', class_="md:text-lg font-semibold mb-4")

    if resume:
        text.append(resume.text)

        paragraphs = soup.find_all('div', class_="ml-2 text-rered markdown text-left tracking-tight font-medium my-4")
        for paragraph in paragraphs:
            text.append(paragraph.text)

        texts.append(" ".join(text))


print(texts)
print(len(texts))


['Protéger Dès lors qu’une catastrophe climatique se produit, il est nécessaire d’apporter une aide d’urgence aux victimes. Les personnes précaires sont d’autant plus impactées par ces drames. Les bénévoles du Secours populaire se mobilisent pour répondre à leurs besoins. L’aide d’urgence prend ensuite la forme d’un accompagnement dans la durée pour aider ces familles à renouer avec leur quotidien. 21,5 millions  de réfugiés climatiques dans le monde ont été recensés en 2022. (Source : Haut Commissariat de l’ONU pour les réfugiés) Inondations, tempêtes, fortes intempéries, grand froid, catastrophes naturelles ou industrielles, incendies… Les conséquences de ces drames sont plus terribles encore chez les familles en situation de pauvreté. Dès qu’un drame humanitaire se produit, un appel aux dons est lancé par le Secours populaire et des aides d’urgence sont apportées au plus vite aux sinistrés. Face aux urgences, le Secours populaire agit en lien avec les pouvoirs publics, les structure

##### Emmaüs Solidarité

Get list of urls for actions

In [75]:
response = requests.get(urls[2])
content = response.content

hrefs2 = []

soup = BeautifulSoup(content, 'html.parser')

for li in soup.find_all('li', id='menu-item-61'):
    for a in li.find_all('a', class_='menu-item menu-item-type-post_type menu-item-object-page')[:-1]:
        href = a.get('href')
        if href:
            hrefs.append(href)
            hrefs2.append(href)
            titles.append(a.text)
            asso.append('Emmaüs Solidarité')

print(len(hrefs))
print(len(titles))

27
27


Getting the text from those urls

In [76]:
for url in hrefs:
    
    text = []
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, 'html.parser')

    # Extracting all paragraphs from the page
    div = soup.find('div', class_="bloc clearFix")

    if div:   
        paragraphes = div.find_all('p')
        lis = div.find_all('li')

        for paragraph in paragraphes:
            text.append(paragraph.text)
        for li in lis:
            text.append(li.text)

        texts.append(" ".join(text))


print(texts)
print(len(texts))


['Protéger Dès lors qu’une catastrophe climatique se produit, il est nécessaire d’apporter une aide d’urgence aux victimes. Les personnes précaires sont d’autant plus impactées par ces drames. Les bénévoles du Secours populaire se mobilisent pour répondre à leurs besoins. L’aide d’urgence prend ensuite la forme d’un accompagnement dans la durée pour aider ces familles à renouer avec leur quotidien. 21,5 millions  de réfugiés climatiques dans le monde ont été recensés en 2022. (Source : Haut Commissariat de l’ONU pour les réfugiés) Inondations, tempêtes, fortes intempéries, grand froid, catastrophes naturelles ou industrielles, incendies… Les conséquences de ces drames sont plus terribles encore chez les familles en situation de pauvreté. Dès qu’un drame humanitaire se produit, un appel aux dons est lancé par le Secours populaire et des aides d’urgence sont apportées au plus vite aux sinistrés. Face aux urgences, le Secours populaire agit en lien avec les pouvoirs publics, les structure

##### WWF

Get list of urls for actions

In [89]:
response = requests.get(f"{urls[3]}champs-daction")
content = response.content

titles2 = []
hrefs2 = []

soup = BeautifulSoup(content, 'html.parser')

for div in soup.find_all('div', class_='inline-teaser-item inline-teaser-item-tile'):
    a = div.find('a')
    href = a.get('href')
    if href:
        hrefs.append(f"{urls[3][:-1:]}{href}")
        hrefs2.append(f"{urls[3][:-1:]}{href}")
        titles.append(div.find('p').text)
        asso.append('WWF')

print(hrefs)
print(titles)

['https://www.secourspopulaire.fr/nos-actions/urgences-climatiques/', 'https://www.secourspopulaire.fr/nos-actions/conflits/', 'https://www.secourspopulaire.fr/nos-actions/aide-migrants-refugies/', 'https://www.secourspopulaire.fr/nos-actions/aide-sans-abri/', 'https://www.secourspopulaire.fr/nos-actions/acces-aux-vacances/', 'https://www.secourspopulaire.fr/nos-actions/accueil-ecoute/', 'https://www.secourspopulaire.fr/nos-actions/aide-alimentaire/', 'https://www.secourspopulaire.fr/nos-actions/aide-materielle/', 'https://www.secourspopulaire.fr/nos-actions/acces-aux-droits/', 'https://www.secourspopulaire.fr/nos-actions/acces-eau-energie/', 'https://www.secourspopulaire.fr/nos-actions/prevention-soins/', 'https://www.secourspopulaire.fr/nos-actions/fracture-numerique/', 'https://www.secourspopulaire.fr/nos-actions/culture-sports-loisirs/', 'https://www.secourspopulaire.fr/nos-actions/acces-education/', 'https://www.secourspopulaire.fr/nos-actions/insertion-socio-professionnelle/', 'h

Getting the text from those urls

In [99]:
texts2 = []
for url in hrefs2:
    
    text = []
    response = requests.get(url)
    content = response.content

    soup = BeautifulSoup(content, 'html.parser')

    # Extracting all paragraphs from the page
    # Recherche de l'élément <h2> dans la division
    div = soup.find('div', class_='paragraph-inner')
    h2_tag = div.find('h2')
    h2_text = h2_tag.text.strip() if h2_tag else 'Pas de texte H2'
    text.append(h2_text)

    # Recherche de tous les paragraphes <p> dans la division
    paragraphs = div.find_all('p')
    p_texts = [p.get_text(separator=" ", strip=True) for p in paragraphs]   
    text.append(" ".join(p_texts))
    one_text = " ".join(text) 


print(texts2)
print(len(texts2))


[]
0


##### Loading all data into the actions dataframe

In [235]:
actions['asso'] = asso
actions['title'] = titles
actions['url'] = hrefs
actions['all_text'] = texts

actions

,asso,title,url,all_text
0,Secours Populaire,Urgences climatiques,https://www.secourspopulaire.fr/nos-actions/ur...,Protéger Dès lors qu’une catastrophe climatiqu...
1,Secours Populaire,Conflits,https://www.secourspopulaire.fr/nos-actions/co...,"Protéger Face aux conflits armés, l’entraide e..."
2,Secours Populaire,Aide aux migrants-réfugiés,https://www.secourspopulaire.fr/nos-actions/ai...,"Protéger Chaque jour, des familles, des enfant..."
3,Secours Populaire,Aide au logement,https://www.secourspopulaire.fr/nos-actions/ai...,"Protéger Dans les permanences d'accueil, de so..."
4,Secours Populaire,Accès aux vacances,https://www.secourspopulaire.fr/nos-actions/ac...,"Émanciper Les vacances, essentielles, sont de ..."
5,Secours Populaire,Accueil et écoute,https://www.secourspopulaire.fr/nos-actions/ac...,Accompagner Les lieux d’accueil du Secours pop...
6,Secours Populaire,Aide alimentaire,https://www.secourspopulaire.fr/nos-actions/ai...,"Accompagner En France, en Europe et partout da..."
7,Secours Populaire,Aide matérielle,https://www.secourspopulaire.fr/nos-actions/ai...,"Accompagner Face à une situation de précarité,..."
8,Secours Populaire,Accès aux droits,https://www.secourspopulaire.fr/nos-actions/ac...,Accompagner Il est fréquent que les personnes ...
9,Secours Populaire,Accès à l’eau et à l’énergie,https://www.secourspopulaire.fr/nos-actions/ac...,Accompagner L’accès à l’eau et à l’énergie est...


In [237]:
actions.to_csv('actions.csv', index=False)